In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_d4.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_b5.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_ea.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_a3.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_9a.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_98.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_ee.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_ca.jpg
/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset/white spot/UNADJUSTEDNONRAW_thumb_88.jpg
/kaggle/input/ident

In [2]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

D0717 04:53:03.319269784      15 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0717 04:53:03.319298316      15 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0717 04:53:03.319302782      15 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0717 04:53:03.319305970      15 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0717 04:53:03.319309256      15 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0717 04:53:03.319312685      15 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0717 04:53:03.319315996      15 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0717 04:53:03.

In [3]:
num_classes = 8

In [4]:
# Create InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False)
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

87910968/87910968 [==============================] - 0s 0us/step


In [5]:
# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [6]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
dataset_dir = '/kaggle/input/identifying-disease-in-tea-leafs/tea sickness dataset'

In [8]:
datagen = ImageDataGenerator(rescale=1./255,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             validation_split=0.2)

In [9]:
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    subset='training')

Found 711 images belonging to 8 classes.


In [10]:
val_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 174 images belonging to 8 classes.


In [11]:
test_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)

Found 885 images belonging to 8 classes.


In [12]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=25,
    validation_data=val_generator,
    validation_steps=len(val_generator))

Epoch 1/25
23/23 [==============================] - 74s 3s/step - loss: 2.0229 - accuracy: 0.4332 - val_loss: 1.2447 - val_accuracy: 0.5345
Epoch 2/25
23/23 [==============================] - 58s 3s/step - loss: 0.8333 - accuracy: 0.7046 - val_loss: 1.0359 - val_accuracy: 0.6207
Epoch 3/25
23/23 [==============================] - 57s 2s/step - loss: 0.6815 - accuracy: 0.7511 - val_loss: 0.9938 - val_accuracy: 0.6322
Epoch 4/25
23/23 [==============================] - 55s 2s/step - loss: 0.5341 - accuracy: 0.8158 - val_loss: 0.9391 - val_accuracy: 0.6207
Epoch 5/25
23/23 [==============================] - 57s 3s/step - loss: 0.5108 - accuracy: 0.8228 - val_loss: 1.0172 - val_accuracy: 0.6379
Epoch 6/25
23/23 [==============================] - 55s 2s/step - loss: 0.3971 - accuracy: 0.8678 - val_loss: 0.8376 - val_accuracy: 0.6954
Epoch 7/25
23/23 [==============================] - 57s 2s/step - loss: 0.3588 - accuracy: 0.8734 - val_loss: 0.8887 - val_accuracy: 0.6207
Epoch 8/25
23/23 [==

In [13]:
#model=model.save('inceptionv3_model.h5')

In [14]:
train_loss, train_acc = model.evaluate(train_generator, steps=len(train_generator))
print(f'Training accuracy: {train_acc}')
print(f'Training loss: {train_loss}')

23/23 [==============================] - 45s 2s/step - loss: 0.1061 - accuracy: 0.9691
Training accuracy: 0.9690576791763306
Training loss: 0.10607416182756424


In [15]:
val_loss, val_acc = model.evaluate(val_generator, steps=len(val_generator))
print(f'Validation accuracy: {val_acc}')
print(f'Validation loss: {val_loss}')

6/6 [==============================] - 11s 2s/step - loss: 0.8879 - accuracy: 0.6897
Validation accuracy: 0.6896551847457886
Validation loss: 0.8878910541534424


In [16]:
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test accuracy: {test_acc}')
print(f'Test loss: {test_loss}')

28/28 [==============================] - 55s 2s/step - loss: 0.2434 - accuracy: 0.9209
Test accuracy: 0.9209039807319641
Test loss: 0.24342432618141174
